<a id='Homework'></a>
# Homework

Theory (5 points):
- Complete theory questions in Google Form
- Take a look at all the links
- Read and analyze all theory `TODO`s

Practice (10 points):
1. Take 2-3 channels from `KyivChannels_Dataset_v01`
2. Apply Clustering OR/AND Topic Modelling techniques to find topics of these channels. Ideal output: `channel_name:[topic_1, topic_2, topic_3]`. Examples: `Крипта Миколи : [криптовалюта, біржа]`
3. (Advanced) Try to come up with a universal approach
4. (Advanced) Apply your approach on other channels

Here is a list of standard topics for TG channels (from TGStat). In the best-case scenario use them as topics.
```
Adult
Art
Blogs
Bookmaking
Books
Business and startups
Career
Courses and guides
Cryptocurrencies
Darknet
Design
Economics
Education
Edutainment
Erotic
Esoterics
Family & Children
Fashion and beauty
Food and cooking
Games
Handiwork
Health and Fitness
Humor and entertainment
Instagram
Interior and construction
Law
Linguistics
Marketing, PR, advertising
Medicine
Music
Nature
News and media
Other
Pictures and photos
Politics
Psychology
Quotes
Religion
Sales
Shock content
Software & Applications
Sport
Technologies
Telegram
Transport
Travel
Video and films
```

# Imports

In [1]:
!pip install -q langid transformers datasets sentence-transformers bertopic

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"]="true"

import pandas as pd
import numpy as np
import nltk
import spacy
import re
import torch
import torch.nn as nn
import string
import langid
import random

from matplotlib import pyplot as plt
from pprint import pprint
from nltk.corpus import stopwords
from nltk import tokenize
from wordcloud import WordCloud, STOPWORDS
from functools import reduce
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from copy import deepcopy
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertModel, pipeline, get_linear_schedule_with_warmup
)
from datasets import Dataset
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import pairwise_distances
from bertopic import BERTopic

torch.manual_seed(42)
torch.backends.cuda.deterministic = True


%matplotlib inline

In [3]:
code_env = 'colab' # 'colab', 'local'

if code_env == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')

    data_path = os.path.join('drive', 'MyDrive', 'MachineLearning', 'Data', 'KyivChannels_Dataset_v01.csv')

elif code_env == 'local':
    data_path = os.path.join('..', 'data', 'KyivChannels_Dataset_v01.csv')


Mounted at /content/drive


# Load Data

In [4]:
df = pd.read_csv(data_path, converters={"Date": pd.to_datetime})

In [5]:
df

,channelname,Date,content,lang
0,kyivpolitics,2023-08-01 09:45:38,Отбой. Угрозы для столицы нет\n\nКиев. Главное...,ru
1,kyivpolitics,2023-08-01 10:03:38,На 8 перекрестках Киева в пилотном режиме внед...,ru
2,kyivpolitics,2023-08-01 14:42:31,⚡️НБУ отозвал банковскую лицензию Конкорд Банк...,ru
3,kyivpolitics,2023-08-01 15:37:34,Завтра синоптики прогнозируют небольшой дождь ...,ru
4,kyivpolitics,2023-08-01 13:06:08,А вот и сам снятый советский герб \n\nКиев. Гл...,ru
...,...,...,...,...
31177,hmarochos,2023-10-27 04:56:20,🎨 Художницю зобовʼязали замалювати мурал на Сі...,uk
31178,hmarochos,2023-10-27 06:12:15,🚧 Львів хоче отримати 50 млн євро на реконстру...,uk
31179,hmarochos,2023-10-27 05:38:42,🙈 На Набережно-Хрещатицькій самовільно влаштув...,uk
31180,semenovatut,2023-10-27 11:50:39,Може залишити Пушкіна?\nБуде об‘єктом перформа...,uk


In [6]:
df['channelname'].value_counts()

channelname
novynylive                     3590
lossolomas_kyiv                3009
darnicalive                    2715
kievvlast                      2273
vichirniykyiv                  1738
big_kyiv                       1670
kyivpolitics                   1383
nashkyivua                     1366
kyiv_novyny_24                 1102
kievreal1                      1096
huevyi_kiev                    1091
obolonlife                     1070
kiev1                          1006
khreschatyk36                   959
kyiv_n                          809
lisovy_masyv_official           722
poznyakyosokorkykharkivskiy     633
hmarochos                       606
ushkiklichko                    578
semenovatut                     526
kyivpasstrans                   362
kyivpatrol                      313
kyivpastrans_live               280
kyivpassengers                  277
uhmc2022                        271
kyiv_pro_office                 248
kyivcityofficial                247
kyiv_by_grishyn 

# Data Processing

Load stopwords from Ukrainian and Russian languages

In [7]:

def read_txt_to_list(path):
    with open(path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

nltk.download("stopwords")

ru_stopwords = stopwords.words("russian")

# Take https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt
!wget https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt
ua_stopwords = read_txt_to_list("stopwords_ua.txt")

combined_stopwords = ru_stopwords + ua_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--2023-11-13 15:15:01--  https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24502 (24K) [text/plain]
Saving to: ‘stopwords_ua.txt’

stopwords_ua.txt    100%[===================>]  23.93K  --.-KB/s    in 0.002s  

2023-11-13 15:15:01 (13.9 MB/s) - ‘stopwords_ua.txt’ saved [24502/24502]



In [8]:
def collapse_dots(input):
    # Collapse sequential dots
    input = re.sub("\.+", ".", input)
    # Collapse dots separated by whitespaces
    all_collapsed = False
    while not all_collapsed:
        output = re.sub(r"\.(( )*)\.", ".", input)
        all_collapsed = input == output
        input = output
    return output


def remove_emojis(text):
    # Define a regex pattern for emojis
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)

    # Remove emojis using the regex pattern
    cleaned_text = emoji_pattern.sub('', text)

    return cleaned_text


def remove_symbols(text):
    # Define a regex pattern for the specified symbols
    symbol_pattern = r'[«»“”—]'

    # Use re.sub to replace the specified symbols with an empty string
    cleaned_text = re.sub(symbol_pattern, '', text)

    return cleaned_text


def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in combined_stopwords]
    return ' '.join(filtered_words)


def process_text(input):
    if isinstance(input, str):
        input = remove_stopwords(input)
        input = remove_emojis(input)
        input = re.sub(r"http\S+", "", input)
        input = re.sub(r"\n+", ". ", input)
        for symb in ["!", ",", ":", ";", "?", "«", "»", "“", "”", "—"]:
            input = re.sub(rf"\{symb}\.", symb, input)
        input = re.sub(r"#\S+", "", input)
        input = collapse_dots(input)
        input = input.strip()
    return input

df["content_processed"] = df["content"].apply(process_text)

In [9]:
df["content"].iloc[2]

'⚡️НБУ отозвал банковскую лицензию Конкорд Банка за нарушение в сфере денежного мониторинга\n\nРанее он фигурировал в расследованиях по мискодингу и махинациям игорного бизнеса. Вывод его с рынка не повлияет на стабильность банковского сектора Украины. Каждый вкладчик банка получит полное возмещение.\n\nКиев. Главное. Политика'

In [10]:
df["content_processed"].iloc[2]

'НБУ отозвал банковскую лицензию Конкорд Банка нарушение сфере денежного мониторинга Ранее фигурировал расследованиях мискодингу махинациям игорного бизнеса. Вывод рынка повлияет стабильность банковского сектора Украины. Каждый вкладчик банка получит полное возмещение. Киев. Главное. Политика'

Drop duplicates and sample texts from 3 channels from whole dataframe

In [11]:
deduplicated_indexes = df.drop_duplicates("content_processed").index

# # Get content from 3 channels - 'novynylive', 'kyivpolitics', 'poznyakyosokorkykharkivskiy'
selected_df = df.loc[deduplicated_indexes].loc[df["channelname"].isin(['novynylive', 'kyivpolitics', 'poznyakyosokorkykharkivskiy'])]


In [12]:
selected_df

,channelname,Date,content,lang,content_processed
0,kyivpolitics,2023-08-01 09:45:38,Отбой. Угрозы для столицы нет\n\nКиев. Главное...,ru,Отбой. Угрозы столицы Киев. Главное. Политика
1,kyivpolitics,2023-08-01 10:03:38,На 8 перекрестках Киева в пилотном режиме внед...,ru,8 перекрестках Киева пилотном режиме внедрят с...
2,kyivpolitics,2023-08-01 14:42:31,⚡️НБУ отозвал банковскую лицензию Конкорд Банк...,ru,НБУ отозвал банковскую лицензию Конкорд Банка ...
3,kyivpolitics,2023-08-01 15:37:34,Завтра синоптики прогнозируют небольшой дождь ...,ru,Завтра синоптики прогнозируют небольшой дождь ...
4,kyivpolitics,2023-08-01 13:06:08,А вот и сам снятый советский герб \n\nКиев. Гл...,ru,снятый советский герб Киев. Главное. Политика
...,...,...,...,...,...
31147,novynylive,2023-10-27 05:28:18,У приміщеннях столичних ТЕЦ ДБР проводить обшу...,uk,приміщеннях столичних ТЕЦ ДБР проводить обшуки...
31148,novynylive,2023-10-27 05:41:29,Окупант захопив безпілотник українських військ...,uk,Окупант захопив безпілотник українських військ...
31149,novynylive,2023-10-27 05:26:00,"Роздали незаконних премій на 1,6 млн: двоє екс...",uk,"Роздали незаконних премій 1,6 млн: двоє експос..."
31150,novynylive,2023-10-27 04:58:00,Німеччина хоче пришвидшити депортацію нелегалі...,uk,Німеччина хоче пришвидшити депортацію нелегалі...


# Topic Modeling

In [13]:
from bertopic import BERTopic
from transformers.pipelines import pipeline

embedding_model = pipeline("feature-extraction", model="distilbert-base-multilingual-cased")
topic_model = BERTopic(embedding_model=embedding_model, verbose=True)


In [14]:
topics, probs = topic_model.fit_transform(selected_df['content_processed'].to_list())

100%|██████████| 5504/5504 [18:00<00:00,  5.09it/s]
2023-11-13 15:34:00,046 - BERTopic - Transformed documents to Embeddings
2023-11-13 15:34:45,397 - BERTopic - Reduced dimensionality
2023-11-13 15:34:45,714 - BERTopic - Clustered reduced embeddings


In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2102,-1_новини_live_киев_политика,"[новини, live, киев, политика, главное, одеса,...",[Окупанти атакували Херсонщину п’яти літаків «...
1,0,651,0_україни_хамас_україні_сша,"[україни, хамас, україні, сша, росії, рф, єс, ...",[Огляд світових ЗМІ вечір: Politico: ЄС зроста...
2,1,390,1_главное_киев_политика_грн,"[главное, киев, политика, грн, это, тыс, будут...","[СБУ разоблачила Киевщине главу ВЛК, которая ""..."
3,2,262,2_окупанти_ова_обстрілу_постраждалих,"[окупанти, ова, обстрілу, постраждалих, пошкод...",[Окупанти скинули авіабомбу міськраду Куп'янсь...
4,3,207,3_україни_сша_ізраїлю_єс,"[україни, сша, ізраїлю, єс, оон, ізраїль, хама...","[ВР розповіли, Україна розпочне переговори вст..."
5,4,131,4_чоловік_поліції_чоловіка_річний,"[чоловік, поліції, чоловіка, річний, волі, заг...","[Одесі підліток зв'язав орендодавця, з'їжджав ..."
6,5,111,5_политика_главное_киев_ещё,"[политика, главное, киев, ещё, кадры, отбой, т...","[Отбой Киев. Главное. Политика, Киев. Главное...."
7,6,109,6_бійці_бригади_відео_зсу,"[бійці, бригади, відео, зсу, окупантів, знищил...",[Воїни 93-ї ОМБр поділилися відеокадрами робот...
8,7,89,7_відео_увага_містить_лексику,"[відео, увага, містить, лексику, ненормативну,...",[Увага! Відео містить ненормативну лексику! Ві...
9,8,83,8_киев_политика_главное_также,"[киев, политика, главное, также, киева, это, у...",[Украинские СМИ продолжают пестрить заголовкам...


In [18]:
topic_model.get_topic(6)

[('бійці', 0.06346711220639283),
 ('бригади', 0.052176667381763624),
 ('відео', 0.051959561332723773),
 ('зсу', 0.049976704284323485),
 ('окупантів', 0.044510861884420466),
 ('знищили', 0.04436727553937708),
 ('омбр', 0.04062445196424579),
 ('слава', 0.038206108735797516),
 ('напрямку', 0.03599833080091178),
 ('показали', 0.03262148051130972)]

In [19]:
topic_model.visualize_topics()

In [53]:
topic_model.save("my_model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


Showcase topics for each channel

In [56]:
loaded_model = BERTopic.load("my_model_dir", embedding_model=embedding_model)

In [61]:
topics_per_class = loaded_model.topics_per_class(selected_df['content_processed'].to_list(),
    classes=selected_df.channelname)

loaded_model.visualize_topics_per_class(topics_per_class,
    top_n_topics=10, normalize_frequency = True)

3it [00:00,  4.88it/s]


In [80]:
# Get a topic for each document in dataframe
selected_df["topic"] = loaded_model.get_document_info(selected_df["content_processed"].to_list())["Topic"].values

In [104]:
# Return top 5 topics for each channel name
grouped_channels = selected_df.loc[selected_df["topic"] != -1].groupby("channelname")["topic"].apply(lambda x: x.value_counts().head(5)).reset_index()

In [106]:
grouped_channels

,channelname,level_1,topic
0,kyivpolitics,1,388
1,kyivpolitics,5,109
2,kyivpolitics,8,82
3,kyivpolitics,16,43
4,kyivpolitics,18,41
5,novynylive,0,584
6,novynylive,2,238
7,novynylive,3,203
8,novynylive,6,107
9,novynylive,4,97


In [114]:
for channel in grouped_channels["channelname"].unique():
    topic_numbers = grouped_channels[grouped_channels["channelname"] == channel]["level_1"].values
    topics = [loaded_model.get_topic(topic_num)[0][0] for topic_num in topic_numbers]
    print(f"{channel} topics: {topics}")

kyivpolitics topics: ['главное', 'политика', 'киев', 'дтп', 'движение']
novynylive topics: ['україни', 'окупанти', 'україни', 'бійці', 'чоловік']
poznyakyosokorkykharkivskiy topics: ['україни', 'відео', 'чоловік', 'загубив', 'окупанти']
